### MAE 3120
## HW 03

Due 03/19

For full credit, show all your work.  

## 1

A signal contains a frequency component at 10,000 Hz with amplitude 0.50 V, but there is also noise in the signal at 60 Hz, amplitude 0.20 V. The DC offset is negligible. The data are acquired with an ADC with the following parameters: input range $\pm 5$ V and 16 bit.

(a) What should be the gain of the amplifier stage(s) to make the best use of the ADC?

- The signal should be amplified to 60-80% of the input of the DAQ system to prevent clipping.  We select a desired voltage of $V_{out} = 4\, \text{V}$ and $V_{in} = 0.5 \, \text{V}$:

The nominal total gain, $G_{tot} = \frac{V_{out}}{V_{in}} = \frac{4}{0.5} = 8$

We have to follow two rules:

1- For the op-amp to operate in its ideal behavior and prevent impedance loading, the gain of each amplifier stage should not exceed $G_{stage} = 30$

2- We should also be concerned by the Gain Bandwidth Product, $GBP$ of the amplifier stages to make sure we have adequate bandwidth.

Here the desired bandwidth is $B = 10,000\,\text{Hz}$.  Since for our op-amps: $GBP = 10^6 \,\text{Hz}$, this forces a maximum gain per amplifier stages of:
\begin{align}
G_{stage} = \frac{GBP}{B} = \frac{10^6}{5\times10^4} = 20
\end{align}

In conclusion, one stage of gain $G_{stage}=G_{tot}=8$ is acceptable.

(b) Select the sampling frequency (justify why).

- To satisfy the Nyquist Criterion, one needs to sample at a frequency $f_s>2\times 10^4\,\text{Hz}$.  For example, $f_s = 25,000\,\text{Hz}$.

(c) What type of filter will you use to filter out the noise?

- The noise is at a lower frequency (60 Hz) as the signal (10,000 Hz).  We will need a high-pass filter to filter out the noise.

(d) Select the appropriate cutoff frequency and order of the filter, justify all your steps.

- As a first guess, one would choose a sampling frequency at least one order of magnitude higher than the noise (600 Hz) and one order of magnitude lower than the carrier frequency (1,000 Hz).  Thus any frequencies between 600 and 1,000 Hz is acceptable.  The goemtric mean of 600 and 1000 is 774, I choose $f_c = 800\,\text{Hz}$.

- The amplified noise should be smaller than the quantization error $Q$.  Amplified noise amplitude is $A_{noise} = 8\times 0.2 = 1.6\,\text{V}$.


In [7]:
import numpy as np

Q=10/2**17
print('Q = ', Q, 'V')
#Gain noise:
Gnoise = Q/1.6
print('Gnoise = ', Gnoise)
# Gain high pass filter
fnoise = 60 # noise frequency
fc = 1000 # cutoff frequency
n = 4 # order of the filter
Ghpf = 1/np.sqrt((1+(fc/fnoise))**(2*n))
print('G_HPF = ', Ghpf)

Q =  7.62939453125e-05 V
Gnoise =  4.76837158203125e-05
G_HPF =  1.0265533875564744e-05


I adjusted the parameters.  Filter of order 6 and with cutoff frequency $f_c = 1,000 Hz$ will have adequate attenuation on the noise.

We now need to check that the carrier frequency has not been distorted, i.e. its phase has not been changed significantly.

In [10]:
fcarrier = 10000
phase = np.arctan(fc/fcarrier)
print('phase = ', 180/np.pi*phase, 'deg')

phase =  5.710593137499643 deg


The phase shift is adequate.

In summary: 
The signal conditioning has an amplifier for Gain 8 (single stage) and high-pass filter of order $n=6$ and cutoff frequency, $f_c = 1,000\,\text{Hz}$.

### 2

(a) How do you detect clipping in your data?

- plot a histogram.  If there are data in the first and/or last bin, there is a risk of clipping.

(b) When acquiring data, how can make sure that there is no aliasing?

- Change the sampling frequency while monitoring the spectra.  If the spectra changes as the frequency changes, then there is aliasing.

(c) In a digital data acquisition system, how many bits are necessary to achieve a resolution of 0.1% of full scale?

- Full scale is the voltage range $V_{max}-V_{min}$.  If one uses the resolution $\Delta V$:

\begin{align}
\Delta V & = \frac{V_{max}-V_{min}}{2^N}\\
\frac{\Delta V}{V_{max}-V_{min}} = 0.001 
\end{align}
Need $N=12\,\text{bits}$

(d) How can you minimize leakage in your spectra?

- 2 strategies: 1- increase frequency resolution (decrease the value of $\Delta f = N/f_s$. i.e. increase $N$ and decrease $f_s$ while respecting the Nyquist Criterion. 2- Perfor windowing on the spectra.


(e) You are to capture the spectra of a 5 Hz signal.  Your sampling frequency is 15 Hz.  Select the number of points, $N$, to acquire to obtain a perfect spectra.

- For a perfect spectra, the signal frequency should be a multiple of the frequency resolution $\Delta f$.  Mathematically:
\begin{align}
f_{signal} = m \Delta f = m \frac{N}{f_s}
\end{align}
For this case $N=3$ is the minimum number of points that would work, but it will only give 3 frequencies in the spectra.  $N=300$ would be more realistic.


(f) In a signal, is there more information in the phase or the amplitude of the harmonics?

- phase.  Remember the Fourier transform on Anthony Hopkins...

### 3

Select the most appropriate digital data acquisition system between the four options below to record the following signal:
\begin{equation*}
y(t) = 2.2 + 3.1 \sin(5,000 \pi t )
\end{equation*}
For full credit, give a critic of each DAS that support your choice.

(a) 16-bit A/D Converter, range =$\pm5$ V, acquisition rate = 20 kS/s.

(b) 12-bit A/D Converter, range = $\pm 10$ V, acquisition rate = 10 kS/s.

(c) 12-bit A/D Converter, range = $\pm 10$ V, acquisition rate = 500 kS/s.

(d) 16-bit A/D Converter range = $\pm 10$ V, acquisition rate = 20 kS/s.

- We need to satisfy three conditions, in this respective order: 1- Sampling rate should be appropriate to satisfy the Nyquist criterion. 2- Choose a DAS that will prevent clipping of the data.  3- Choose the system with the highest resolution (lowest $Q$ or $\Delta V$).

1- The signal frequency is $f=2,500 Hz$, to satisfy the Nyquist Criterion, we need to acquire data at $f_s > 5,000\,\text{Hz}$.  All the DAS are appropriate for this criterion.

2- The signal amplitude is in the range: $[-0.9 - 5.3] \text{V}$.  DAS (a) will result in clipping.  (b), (c), (d) are acceptable.

3- DAS (d) has the smallest $Q$ and is the best option.